### Simualting BlackJack

In [38]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [45]:
ceil_to_10 = lambda x: 10 if x>10 else x
def basic_strategy_rules(player, dealer, doubling_down=True):
    """
    Apply basic strategy rules.
    Details: http://www.cs.bu.edu/~hwxi/academic/courses/CS320/Spring02/assignments/06/basic-strategy.html
    player: player cards ceiled to 10
    dealer: dealer card ceiled to 10
    """
    summe = sum(player)
    if player[0] != player[1]:
        if 1 not in player:
            if summe <= 8:
                return "hit"
            elif summe == 9:
                if dealer >= 3 and dealer <= 6:
                    return "double"
                return "hit"
            elif summe == 10:
                if dealer >= 2 and dealer <= 9:
                    return "double"
                return "hit"
            elif summe == 11:
                if dealer >= 2 and dealer <= 10:
                    return "double"
                return "hit"
            elif summe == 12:
                if dealer >= 4 and dealer <= 6:
                    return "stand"
                return "hit"
            elif summe >= 13 and summe <= 16:
                if dealer >= 2 and dealer <= 6:
                    return "stand"
                return "hit"
            return "stand"
        else:
            if summe == 3 or summe == 4:
                if dealer == 5 or dealer == 6:
                    return "double"
                return "hit"
            elif summe == 5 or summe == 6:
                if dealer >= 4 and dealer <= 6:
                    return "double"
                return "hit"
            elif summe == 7:
                if dealer >= 3 and dealer <= 6:
                    return "double"
                return "hit"
            elif summe == 8:
                if dealer >= 3 and dealer <= 6:
                    return "double"
                elif dealer == 2 or dealer == 7 or dealer == 8:
                    return "stand"
                return "hit"
            else:
                return "stand"
    else:
        if doubling_down:
            if player[0] == 1:
                return "split"
            elif player[0] == 2 or player[0] == 3:
                if dealer >= 2 and dealer <= 7:
                    return "split"
                return "hit"
            elif player[0] == 4:
                if dealer == 5 or dealer == 6:
                    return "split"
                return "hit"
            elif player[0] == 5:
                if dealer >= 2 and dealer <= 9:
                    return "double"
                return "hit"
            elif player[0] == 6 or player[0] == 7:
                if dealer >= 2 and dealer <= player[0]:
                    return "split"
                return "hit"
            elif player[0] == 8:
                return "split"
            elif player[0] == 9:
                if dealer == 1 or dealer == 7 or dealer == 10:
                    return "stand"
                return "split"
            return "stand"
        else:
            if player[0] == 2 or player[0] == 3:
                if dealer >= 4 and dealer <= 7:
                    return "split"
                return "hit"
            elif player[0] == 4:
                return "hit"
            elif player[0] == 6:
                if dealer >= 3 and dealer <= 6:
                    return "split"
                return "hit"

In [56]:
# parameters
N = 4 # number of decks
money = 100 # entry money
bet = 1 # minimum bet
deck = np.repeat(np.arange(1,14), 4*N)
np.random.shuffle(deck) # inline shuffle
print(deck)
cut_card = np.random.randint(low=int(len(deck)/2), high=int(len(deck)))
print(len(deck))
print(cut_card)

[11  3  7  6 13  6  5  4  3  3 11 12  8  1 12  2  5 10 12 13  6  3  5 13  5
  1  3  6  2  3  2  5 11 11  2  5  9  3 12  1  2  2  8  1  8 12 10  8  2  4
  2 13  6  4 12  4 10 10 12  1 11  9  4 10  1  4  2 13 10 13  5  6 12  6  2
  8  9 13  1 11  5  8  9  5 11  2  3 10  6  4 12 10  3  4  5 12 10  5  7 13
  6  1  6  7 13  8  9 10  9 10  4  3  7 11  2  8  2 10  9  9  5 11 12  8  9
  9 12  8 13  8  8  3  1 10  8 11  5  2  7  4  5  1  5  4  3  7  8  7  3 13
 13 12 13 11  6  7  7  4  1  1  9  7  1  6 10  9  1 13 12  7  7 11  6  4  1
  2  7  6  8 13  4 11  9  6  9 12  3  4  7 11  7 10  4  9 10 12  1  8  2  7
  5  3 13  9  6 11  3 11]
208
146


In [54]:
# main loop
i = 0
counter = 0
while cut_card>0 and i<len(deck):
    counter += 1
    bet = 1
    player = deck[i:i+2]
    dealer = deck[i+2]
    i = i+3
    cut_card -= 3
    print(basic_strategy_rules(player, dealer))
print(counter)
print(i)
print(cut_card)
# player = deck[:2]
# dealer = deck[2:4]
# print(player)
# print(dealer)
# vfunc = np.vectorize(ceil210)
# vfunc(player)

stand
hit
hit
hit
hit
split
stand
stand
hit
hit
stand
hit
stand
stand
hit
stand
double
stand
hit
hit
stand
hit
hit
stand
stand
hit
hit
hit
stand
hit
stand
hit
stand
stand
stand
stand
hit
stand
hit


In [24]:
print()

array([ 1, 13])

In [52]:
dealer = deck[2:4]
print(dealer)
dealer = np.append(dealer, deck[5])

[10  8]


In [53]:
dealer

array([10,  8,  8])